In [1]:
import pandas as pd 
import numpy as np

import requests

#!pip install folium
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

In [2]:
df = pd.read_csv("Toronto_df_with_ll.csv", index_col = 0)
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Keep only those boroughs that have the work Toronto in them

In [3]:
df = df[df["Borough"].str.contains('Toronto')]
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [4]:
df.shape

(39, 5)

After loading the data lets get down to visualizing it, since the properties of the data are well-known from the previous two assignments.

In [5]:
lat_toronto = 43.653908
lng_toronto = -79.384293

toronto_map = folium.Map(location = [lat_toronto, lng_toronto], zoom_start = 12)

for lat, lng, borough, neighborhood in zip(df.Latitude, df.Longitude, df.Borough, df.Neighborhood):
    label = '{}, {}'.format(borough, neighborhood)
    label = folium.Popup(label, parse_html = False)
    folium.CircleMarker(location = [lat, lng], radius = 5, popup = label, color = 'purple', 
                        fill = True, fill_color = 'pink', fill_opacity = 0.7).add_to(toronto_map)
toronto_map

Now let's obtain the data from forsquare!
We will explore the venues :)

In [6]:
# Foursquare Credentials
CLIENT_ID = 'PY1CS2HDSFYG05EEGYLLTW0FNAVVABZLW3LNSEHENGCQOKG2' # your Foursquare ID
CLIENT_SECRET = 'O35UDS3VRE0L1CSGACLLNVVP3FNIMNSMBPBKOZQAQWWFMWCD' # your Foursquare Secret
VERSION = '20203004' # Foursquare API version

Let's create a function to repeat the same process to all the neighborhoods

In [7]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [8]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North &amp; West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James T

In [9]:
print(toronto_venues.shape)
toronto_venues.head()

(1604, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


Let's check how many venues were returned for each neighborhood

In [10]:
toronto_venues.groupby('Neighborhood', as_index=False).count()[:5]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,55,55,55,55,55,55
1,"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
2,Business reply mail Processing CentrE,16,16,16,16,16,16
3,"CN Tower, King and Spadina, Railway Lands, Har...",16,16,16,16,16,16
4,Central Bay Street,60,60,60,60,60,60


Let's do the same with the venues :)

In [11]:
toronto_venues.groupby('Venue Category', as_index=False).count().tail()

,Venue Category,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
223,Video Game Store,2,2,2,2,2,2
224,Vietnamese Restaurant,7,7,7,7,7,7
225,Wine Bar,8,8,8,8,8,8
226,Women's Store,1,1,1,1,1,1
227,Yoga Studio,11,11,11,11,11,11


Get the number of the unique venues:

In [12]:
toronto_venues['Venue Category'].unique().shape[0]

228

Get the first 10 most frequent types of venues

In [13]:
toronto_venues['Venue Category'].value_counts()[:10]

Coffee Shop            153
Café                    90
Restaurant              56
Italian Restaurant      41
Hotel                   39
Japanese Restaurant     34
Park                    33
Bakery                  30
Bar                     28
Gym                     26
Name: Venue Category, dtype: int64

## Let's analyze each neighborhood

In [14]:
# one hot encoding
onehot_toronto = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot_toronto['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot_toronto.columns[-1]] + list(onehot_toronto.columns[:-1])
onehot_toronto = onehot_toronto[fixed_columns]

onehot_toronto.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
print("We get total number of samples (rows): %d and features (columns): %d" %(onehot_toronto.shape[0], onehot_toronto.shape[1]))

We get total number of samples (rows): 1604 and features (columns): 228


Let's group the new df by neighborhood and apply the mean function

In [16]:
grouped = onehot_toronto.groupby('Neighborhood', as_index = False).mean()
grouped.head(2)

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.018182,0.0,0.0,0.0,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [17]:
grouped.shape

(39, 228)

It turns out that we are having 39 neighbours and 228 features.

We will obtain information about each neighborhood and it's top 5 venues

In [18]:
num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2                Café  0.04
3  Seafood Restaurant  0.04
4         Cheese Shop  0.04


----Brockton, Parkdale Village, Exhibition Place----
               venue  freq
0               Café  0.12
1          Nightclub  0.12
2     Breakfast Spot  0.08
3        Coffee Shop  0.08
4  Convenience Store  0.04


----Business reply mail Processing CentrE----
                  venue  freq
0  Gym / Fitness Center  0.06
1         Garden Center  0.06
2            Comic Shop  0.06
3           Pizza Place  0.06
4      Recording Studio  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.19
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3   Harbor / Marina  0.06
4           Airport  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.18
1  Italia

let's write a function to sort the venues in descending order.

In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Let's create the new dataframe and display the top 10 venues for each neighborhood.

In [20]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(3)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Bakery,Seafood Restaurant,Restaurant,Cheese Shop,Café,Jazz Club,Shopping Mall
1,"Brockton, Parkdale Village, Exhibition Place",Nightclub,Café,Breakfast Spot,Coffee Shop,Climbing Gym,Bar,Stadium,Bakery,Intersection,Italian Restaurant
2,Business reply mail Processing CentrE,Park,Auto Workshop,Comic Shop,Pizza Place,Recording Studio,Burrito Place,Restaurant,Brewery,Light Rail Station,Smoke Shop


### Cluster Neighborhoods into 5 Clusters

In [21]:
# set number of clusters
kclusters = 5

X = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(X)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 4, 0, 1, 0,
       0, 0, 0, 0, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [22]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Pub,Park,Café,Breakfast Spot,Theater,Ice Cream Shop,Chocolate Shop,Spa
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Diner,Yoga Studio,Creperie,Sandwich Place,Burger Joint,Burrito Place,Café,Park
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Restaurant,Japanese Restaurant,Bubble Tea Shop,Cosmetics Shop,Ramen Restaurant,Bookstore
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Café,Coffee Shop,Cocktail Bar,Gastropub,American Restaurant,Hotel,Creperie,Lingerie Store,Department Store,Moroccan Restaurant
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Coffee Shop,Pub,Trail,Women's Store,Deli / Bodega,Department Store,Dessert Shop,Diner,Distribution Center


In [23]:
# create map
map_clusters = folium.Map(location=[lat_toronto, lng_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged['Neighborhood'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine the 5 Clusters

In [24]:
merged.loc[merged['Cluster Labels'] == 0, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,0,Coffee Shop,Bakery,Pub,Park,Café,Breakfast Spot,Theater,Ice Cream Shop,Chocolate Shop,Spa
4,Downtown Toronto,0,Coffee Shop,Sushi Restaurant,Diner,Yoga Studio,Creperie,Sandwich Place,Burger Joint,Burrito Place,Café,Park
9,Downtown Toronto,0,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Restaurant,Japanese Restaurant,Bubble Tea Shop,Cosmetics Shop,Ramen Restaurant,Bookstore
15,Downtown Toronto,0,Café,Coffee Shop,Cocktail Bar,Gastropub,American Restaurant,Hotel,Creperie,Lingerie Store,Department Store,Moroccan Restaurant
19,East Toronto,0,Health Food Store,Coffee Shop,Pub,Trail,Women's Store,Deli / Bodega,Department Store,Dessert Shop,Diner,Distribution Center
20,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Beer Bar,Bakery,Seafood Restaurant,Restaurant,Cheese Shop,Café,Jazz Club,Shopping Mall
24,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Burger Joint,Bar,Ice Cream Shop,Salad Place,Thai Restaurant,Bubble Tea Shop
25,Downtown Toronto,0,Grocery Store,Café,Park,Baby Store,Italian Restaurant,Diner,Restaurant,Candy Store,Nightclub,Coffee Shop
30,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Gym,Clothing Store,Hotel,Thai Restaurant,Deli / Bodega,Concert Hall,American Restaurant
31,West Toronto,0,Bakery,Pharmacy,Middle Eastern Restaurant,Art Gallery,Liquor Store,Café,Bar,Bank,Pool,Supermarket


In [25]:
merged.loc[merged['Cluster Labels'] == 1, merged.columns[[1] + list(range(5, merged.shape[1]))]][:3]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,Central Toronto,1,Playground,Women's Store,Dance Studio,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


In [26]:
merged.loc[merged['Cluster Labels'] == 2, merged.columns[[1] + list(range(5, merged.shape[1]))]][:3]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,Central Toronto,2,Park,Sushi Restaurant,Jewelry Store,Trail,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
91,Downtown Toronto,2,Park,Trail,Playground,Cuban Restaurant,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


In [27]:
merged.loc[merged['Cluster Labels'] == 3, merged.columns[[3] + list(range(5, merged.shape[1]))]][:3]

,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,43.711695,3,Garden,Ice Cream Shop,Women's Store,Dance Studio,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


In [28]:
merged.loc[merged['Cluster Labels'] == 4, merged.columns[[1] + list(range(5, merged.shape[1]))]][:3]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,Central Toronto,4,Swim School,Bus Line,Park,College Rec Center,College Gym,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run


## Notes: 

Coffee shop, groceries and restaurants are grouped together.  
Then, playgrouynds, after that parks, gardens and finally, swim schools. 